# Extracting Database Schema from PostgreSQL

In [2]:
import psycopg2
import json



# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="123456",
    host="localhost",
    port="5432"
)


cursor = conn.cursor()

# Get tables
cursor.execute("""
    SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';
""")
tables = [row[0] for row in cursor.fetchall()]

# Get columns
schema = []
for table in tables:
    cursor.execute(f"""
        SELECT column_name, data_type 
        FROM information_schema.columns 
        WHERE table_name = '{table}';
    """)
    columns = [{"name": row[0], "type": row[1]} for row in cursor.fetchall()]
    
    schema.append({"name": table, "columns": columns})

# Get foreign keys
cursor.execute("""
    SELECT 
        conrelid::regclass AS table_name, 
        a.attname AS column_name,
        confrelid::regclass AS foreign_table_name
    FROM pg_constraint AS c
    JOIN pg_attribute AS a 
    ON a.attnum = ANY(c.conkey) AND a.attrelid = c.conrelid
    WHERE c.contype = 'f';
""")
foreign_keys = cursor.fetchall()
for fk in foreign_keys:
    for table in schema:
        if table["name"] == fk[0]:
            for column in table["columns"]:
                if column["name"] == fk[1]:
                    column["foreign_key"] = f"{fk[2]}.{fk[1]}"

# Close connection
cursor.close()
conn.close()

# Output schema as JSON
schema_json = json.dumps({"tables": schema}, indent=4)
print(schema_json)


{
    "tables": [
        {
            "name": "packages_apt_postgresql_org",
            "columns": [
                {
                    "name": "id",
                    "type": "integer"
                },
                {
                    "name": "last_updated",
                    "type": "timestamp without time zone"
                },
                {
                    "name": "aptdata",
                    "type": "jsonb"
                }
            ]
        },
        {
            "name": "packages_yum_postgresql_org",
            "columns": [
                {
                    "name": "id",
                    "type": "integer"
                },
                {
                    "name": "last_updated",
                    "type": "timestamp without time zone"
                },
                {
                    "name": "yumdata",
                    "type": "jsonb"
                }
            ]
        },
        {
            "name": "film_actor"

# Creating Summary Metadata

In [5]:
import os
from google import genai
from google.genai import types
from dotenv import load_dotenv
from database import *
load_dotenv()

True

In [6]:
client = genai.Client(api_key=os.getenv('API_KEY'))